In [60]:
# Install liberaries
!pip install pydrive
!pip install openpyxl

In [0]:
# Data manipulation
import pandas as pd
import numpy as np

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()

drive = GoogleDrive(gauth)

train_downloaded = drive.CreateFile({'id': '1oZ_vH6KpHm7zaPl4X9KXytfl3XhA9QYb'})
train_downloaded.GetContentFile('train.csv')

test_downloaded = drive.CreateFile({'id': '1R9SeLlmQm6kKnsSM72xNCw07VrG7I6Od'})
test_downloaded.GetContentFile('test.csv')

# CSV files to pandas DataFrame
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [0]:
# Format data
tr_gender_list = []
tr_embarked_list = []
tr_cabin_list = []

ts_gender_list = []
ts_embarked_list = []
ts_cabin_list = []

for index, row in df_train.iterrows():
  tr_gender_list.append(1 if row['Sex'] == 'male' else 0)
  tr_embarked_list.append(0 if row['Embarked'] == 'Q' else 1 if row['Embarked'] == 'S' else 2)
  tr_cabin_list.append(0 if row['Cabin'] == 'NaN' else 1)

for index, row in df_test.iterrows():
  ts_gender_list.append(1 if row['Sex'] == 'male' else 0)
  ts_embarked_list.append(0 if row['Embarked'] == 'Q' else 1 if row['Embarked'] == 'S' else 2)
  ts_cabin_list.append(0 if row['Cabin'] == 'NaN' else 1)
  
df_train['Sex'] = tr_gender_list
df_train['Embarked'] = tr_embarked_list
df_train['Cabin'] = tr_cabin_list

df_test['Sex'] = ts_gender_list
df_test['Embarked'] = ts_embarked_list
df_test['Cabin'] = ts_cabin_list


In [64]:
#print(df_train.head(1))
print(df_train.isnull().any())

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin          False
Embarked       False
dtype: bool


**Features: **

> input-size=(891, 10) [df_train]
> input-size=(418, 10) [df_test]
  


In [65]:
from keras.models import Sequential
from keras.layers import Dense

Xtr = df_train.drop(['PassengerId', 'Survived', 'Name', 'Ticket', 'Age'], axis=1)
Ytr = df_train['Survived']

Xts = df_test.drop(['PassengerId', 'Name', 'Ticket', 'Age'], axis=1)

model = Sequential()
model.add(Dense(14, input_dim=7, activation='relu'))
model.add(Dense(14, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
model.fit(Xtr, Ytr, epochs=500, verbose=1)

Epoch 1/500
891/891 [==============================] - 2s 2ms/step - loss: 0.4910 - acc: 0.3838
Epoch 2/500
891/891 [==============================] - 0s 77us/step - loss: 0.2617 - acc: 0.5511
Epoch 3/500
891/891 [==============================] - 0s 76us/step - loss: 0.1998 - acc: 0.7093
Epoch 4/500
891/891 [==============================] - 0s 71us/step - loss: 0.1929 - acc: 0.7082
Epoch 5/500
891/891 [==============================] - 0s 74us/step - loss: 0.1906 - acc: 0.7082
Epoch 6/500
891/891 [==============================] - 0s 76us/step - loss: 0.1862 - acc: 0.7059
Epoch 7/500
891/891 [==============================] - 0s 78us/step - loss: 0.1821 - acc: 0.7172
Epoch 8/500
891/891 [==============================] - 0s 83us/step - loss: 0.1808 - acc: 0.7284
Epoch 9/500
891/891 [==============================] - 0s 62us/step - loss: 0.1785 - acc: 0.7318
Epoch 10/500
891/891 [==============================] - 0s 66us/step - loss: 0.1778 - acc: 0.7407
Epoch 11/500
891/891 [========

In [66]:
pred = model.predict(Xts)
result = np.around(pred, 0)

tdf = pd.DataFrame()
tdf['PassengerId'] = df_test['PassengerId']
tdf['Survived'] = result

tdf = tdf.fillna(0)
tdf

,PassengerId,Survived
0,892,0.0
1,893,0.0
2,894,0.0
3,895,0.0
4,896,1.0
5,897,0.0
6,898,1.0
7,899,0.0
8,900,1.0
9,901,0.0


In [0]:
from openpyxl.workbook import Workbook
from openpyxl.writer.excel import save_virtual_workbook

auth.authenticate_user()

wb = Workbook()
writer = pd.ExcelWriter('', engine='openpyxl')
writer.book = wb
tdf.to_excel(writer)

f = drive.CreateFile({'id': '1hM0nNt2o84nWTtLmbJAtnDWjBonxhasg'}) #submission.csv
f.GetContentFile('submission.csv')

with open('submission.csv', 'a') as file:
    tdf.to_csv(file, index=False, header=True)

f.SetContentFile('submission.csv')
f.Upload()